In [30]:
import sqlite3
import re, time, requests
from bs4 import BeautifulSoup
from urllib.parse import quote
conn = sqlite3.connect('databases/pokedex.db')
conn.row_factory = sqlite3.Row
c = conn.cursor()


In [31]:
BASE_URL = "https://bulbapedia.bulbagarden.net/wiki/{}_({})"

In [32]:
STAT_ORDER = ['hp', 'attack', 'defense', 'sp_atk', 'sp_def', 'speed']

In [33]:
c.execute('''CREATE TABLE IF NOT EXISTS form_stats_overrides (
  dex_no      INTEGER NOT NULL REFERENCES pokemon(dex_no),
  form_code   TEXT NOT NULL,
  hp          SMALLINT,
  attack      SMALLINT,
  defense     SMALLINT,
  sp_atk      SMALLINT,
  sp_def      SMALLINT,
  speed       SMALLINT,
  PRIMARY KEY (dex_no, form_code)
);''')

In [34]:
c.execute('''INSERT OR REPLACE INTO form_stats_overrides (dex_no, form_code, hp, attack, defense, sp_atk, sp_def, speed) VALUES (?,?,?,?,?,?,?,?)''', (383, 'Primal', 100, 180, 160, 150, 90, 90))
conn.commit()

In [47]:
c.execute('''SELECT * FROM form_stats_overrides''')
rows = c.fetchall()
for row in rows:
    print(row['dex_no'], row['form_code'], [row[stat] for stat in STAT_ORDER])

128 Paldea [75, 110, 105, 30, 70, 100]
382 Primal [100, 150, 90, 180, 160, 90]
3 Mega [80, 100, 123, 122, 120, 80]
6 Mega X [78, 130, 111, 130, 85, 100]
6 Mega Y [78, 104, 78, 159, 115, 100]
9 Mega [79, 103, 120, 135, 115, 78]
15 Mega [65, 150, 40, 15, 80, 145]
18 Mega [83, 80, 80, 135, 80, 121]
65 Mega [55, 50, 65, 175, 95, 150]
80 Mega [95, 75, 180, 130, 80, 30]
94 Mega [60, 65, 80, 170, 95, 130]
115 Mega [105, 125, 100, 60, 100, 100]
127 Mega [65, 155, 120, 65, 90, 105]
130 Mega [95, 155, 109, 70, 130, 81]
142 Mega [80, 135, 85, 70, 95, 150]
150 Mega X [106, 190, 100, 154, 100, 130]
150 Mega Y [106, 150, 70, 194, 120, 140]
181 Mega [90, 95, 105, 165, 110, 45]
208 Mega [75, 125, 230, 55, 95, 30]
212 Mega [70, 150, 140, 65, 100, 75]
214 Mega [80, 185, 115, 40, 105, 75]
229 Mega [75, 90, 90, 140, 90, 115]
248 Mega [100, 164, 150, 95, 120, 71]
254 Mega [70, 110, 75, 145, 85, 145]
257 Mega [80, 160, 80, 130, 80, 100]
260 Mega [100, 150, 110, 95, 110, 70]
282 Mega [68, 85, 65, 165, 135, 1

In [36]:
import re
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup

PAGES = [
    "https://www.serebii.net/pokedex-xy/megaevolution.shtml",
    "https://www.serebii.net/omegarubyalphasapphire/megaevolutions.shtml",
    "https://www.serebii.net/sunmoon/megaevolutions.shtml",
    "https://www.serebii.net/letsgopikachueevee/megaevolution.shtml",
    "https://www.serebii.net/legendsz-a/megaevolutions.shtml",
]

CANON = ["HP","Att","Def","S.Att","S.Def","Spd"]
HEADER_ALIASES = {
    "HP": {"HP"},
    "Att": {"Att","Atk","Attack"},
    "Def": {"Def","Defense"},
    "S.Att": {"S.Att","Sp. Atk","SpAtk","Sp Atk"},
    "S.Def": {"S.Def","Sp. Def","SpDef","Sp Def"},
    "Spd": {"Spd","Spe","Speed"},
}
name_re = re.compile(r"^Mega [A-Za-z0-9 .'\-♀♂]+$")

def norm(s): return re.sub(r"\s+", " ", (s or "").strip())
def cell_text(el): return norm(el.get_text(" ", strip=True)) if el else ""

def normalize_header(h):
    h = norm(h)
    for canon, alts in HEADER_ALIASES.items():
        if h in alts: return canon
    return h

def has_stat_header(table):
    # Look at first few rows for a 6-cell header matching CANON in order
    for tr in table.find_all("tr")[:4]:
        cells = [normalize_header(cell_text(c)) for c in tr.find_all(["th","td"])]
        for i in range(len(cells)-5):
            if cells[i:i+6] == CANON:
                return i  # return the starting index of HP
    return None

def extract_row_stats(txt):
    nums = [int(x) for x in re.findall(r"\b\d+\b", txt)]
    if len(nums) < 6: 
        return None
    stats = nums[-6:]
    # sanity: base stats should be 1..255
    if all(1 <= n <= 255 for n in stats):
        return stats
    return None

def parse_megas_from_page(url) -> Dict[str, List[int]]:
    resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    resp.encoding = resp.apparent_encoding or resp.encoding
    soup = BeautifulSoup(resp.text, "lxml")

    out = {}
    for table in soup.find_all("table"):
        hp_idx = has_stat_header(table)
        if hp_idx is None:
            continue  # ignore non-stat tables entirely

        for tr in table.find_all("tr")[1:]:  # skip header row
            cells = tr.find_all(["td","th"])
            if not cells: 
                continue
            # find the name cell
            name = None
            for c in cells:
                t = cell_text(c)
                if t.startswith("Mega ") and name_re.match(t):
                    name = t
                    break
            if not name:
                continue

            stats = extract_row_stats(cell_text(tr))
            if not stats:
                continue

            # Store
            out[name] = stats
    return out

# aggregate + dedupe
mega_stats: Dict[str, List[int]] = {}
for url in PAGES:
    for k, v in parse_megas_from_page(url).items():
        mega_stats.setdefault(k, v)
        
len(mega_stats)


75

In [37]:
mega_stats

{'Mega Venusaur': [80, 100, 123, 122, 120, 80],
 'Mega Charizard X': [78, 130, 111, 130, 85, 100],
 'Mega Charizard Y': [78, 104, 78, 159, 115, 100],
 'Mega Blastoise': [79, 103, 120, 135, 115, 78],
 'Mega Beedrill': [65, 150, 40, 15, 80, 145],
 'Mega Pidgeot': [83, 80, 80, 135, 80, 121],
 'Mega Alakazam': [55, 50, 65, 175, 95, 150],
 'Mega Slowbro': [95, 75, 180, 130, 80, 30],
 'Mega Gengar': [60, 65, 80, 170, 95, 130],
 'Mega Kangaskhan': [105, 125, 100, 60, 100, 100],
 'Mega Pinsir': [65, 155, 120, 65, 90, 105],
 'Mega Gyarados': [95, 155, 109, 70, 130, 81],
 'Mega Aerodactyl': [80, 135, 85, 70, 95, 150],
 'Mega Mewtwo X': [106, 190, 100, 154, 100, 130],
 'Mega Mewtwo Y': [106, 150, 70, 194, 120, 140],
 'Mega Ampharos': [90, 95, 105, 165, 110, 45],
 'Mega Steelix': [75, 125, 230, 55, 95, 30],
 'Mega Scizor': [70, 150, 140, 65, 100, 75],
 'Mega Heracross': [80, 185, 115, 40, 105, 75],
 'Mega Houndoom': [75, 90, 90, 140, 90, 115],
 'Mega Tyranitar': [100, 164, 150, 95, 120, 71],
 'Meg

In [38]:
def base_name(mega: str) -> str:
    # Mega Charizard X -> Charizard
    # Mega Charizard Y -> Charizard
    name = mega.replace("Mega ", "")
    if name.endswith(" X") or name.endswith(" Y"):
        name = name[:-2]
    return name

ids = []
for mega in mega_stats.keys():
    base = base_name(mega)
    c.execute("SELECT id FROM pokemon WHERE name = ?", (base,))
    row = c.fetchone()
    ids.append(row["id"] if row else None)  # keep position even if missing



In [39]:
megas_list =  zip(ids, mega_stats.items())
for item in megas_list:
    print(item)

(3, ('Mega Venusaur', [80, 100, 123, 122, 120, 80]))
(6, ('Mega Charizard X', [78, 130, 111, 130, 85, 100]))
(6, ('Mega Charizard Y', [78, 104, 78, 159, 115, 100]))
(9, ('Mega Blastoise', [79, 103, 120, 135, 115, 78]))
(15, ('Mega Beedrill', [65, 150, 40, 15, 80, 145]))
(18, ('Mega Pidgeot', [83, 80, 80, 135, 80, 121]))
(65, ('Mega Alakazam', [55, 50, 65, 175, 95, 150]))
(80, ('Mega Slowbro', [95, 75, 180, 130, 80, 30]))
(94, ('Mega Gengar', [60, 65, 80, 170, 95, 130]))
(115, ('Mega Kangaskhan', [105, 125, 100, 60, 100, 100]))
(127, ('Mega Pinsir', [65, 155, 120, 65, 90, 105]))
(130, ('Mega Gyarados', [95, 155, 109, 70, 130, 81]))
(142, ('Mega Aerodactyl', [80, 135, 85, 70, 95, 150]))
(150, ('Mega Mewtwo X', [106, 190, 100, 154, 100, 130]))
(150, ('Mega Mewtwo Y', [106, 150, 70, 194, 120, 140]))
(181, ('Mega Ampharos', [90, 95, 105, 165, 110, 45]))
(208, ('Mega Steelix', [75, 125, 230, 55, 95, 30]))
(212, ('Mega Scizor', [70, 150, 140, 65, 100, 75]))
(214, ('Mega Heracross', [80, 185, 

In [40]:
# ids must align with mega_stats.keys() order (don’t use the dedupbed unique_ids here)
megas_list = [(pid, item) for pid, item in zip(ids, mega_stats.items()) if pid is not None]

print("len(ids) =", len(ids))
print("len(mega_stats) =", len(mega_stats))
print("len(megas_list) =", len(megas_list))
print("sample:", megas_list[:2])


len(ids) = 75
len(mega_stats) = 75
len(megas_list) = 74
sample: [(3, ('Mega Venusaur', [80, 100, 123, 122, 120, 80])), (6, ('Mega Charizard X', [78, 130, 111, 130, 85, 100]))]


In [41]:
# Ensure a conflict target so updates actually UPDATE
c.execute("""
CREATE TABLE IF NOT EXISTS form_stats_overrides (
  dex_no    INTEGER NOT NULL,
  form_code TEXT    NOT NULL,
  hp        INTEGER NOT NULL,
  attack    INTEGER NOT NULL,
  defense   INTEGER NOT NULL,
  sp_atk    INTEGER NOT NULL,
  sp_def    INTEGER NOT NULL,
  speed     INTEGER NOT NULL,
  PRIMARY KEY (dex_no, form_code)
)
""")
c.execute("CREATE UNIQUE INDEX IF NOT EXISTS u_form ON form_stats_overrides(dex_no, form_code)")

rows = []
for dex_id, (mega_name, stats) in megas_list:
    if not stats or len(stats) != 6:
        continue
    form_code = "Mega X" if mega_name.endswith(" X") else "Mega Y" if mega_name.endswith(" Y") else "Mega"
    hp, atk, defe, spa, spd, spe = map(int, stats)
    rows.append((int(dex_id), form_code, hp, atk, defe, spa, spd, spe))

print("Prepared rows:", len(rows))
if rows:
    before = conn.total_changes
    c.executemany("""
    INSERT INTO form_stats_overrides
      (dex_no, form_code, hp, attack, defense, sp_atk, sp_def, speed)
    VALUES (?,?,?,?,?,?,?,?)
    ON CONFLICT(dex_no, form_code) DO UPDATE SET
      hp=excluded.hp,
      attack=excluded.attack,
      defense=excluded.defense,
      sp_atk=excluded.sp_atk,
      sp_def=excluded.sp_def,
      speed=excluded.speed
    """, rows)
    conn.commit()
    print("Rows changed:", conn.total_changes - before)
else:
    print("Nothing to write: rows list is empty.")


Prepared rows: 74
Rows changed: 74


In [42]:
print("pokemon schema:")
c.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='pokemon'")
print(c.fetchone()[0], "\n")

print("form_stats_overrides schema:")
c.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='form_stats_overrides'")
print(c.fetchone()[0], "\n")

print("FKs on form_stats_overrides:")
c.execute("PRAGMA foreign_key_list(form_stats_overrides)")
print(c.fetchall())


pokemon schema:
CREATE TABLE pokemon (id INTEGER PRIMARY KEY AUTOINCREMENT,name TEXT NOT NULL UNIQUE) 

form_stats_overrides schema:
CREATE TABLE form_stats_overrides (
  dex_no    INTEGER NOT NULL,
  form_code TEXT    NOT NULL,
  hp        INTEGER NOT NULL,
  attack    INTEGER NOT NULL,
  defense   INTEGER NOT NULL,
  sp_atk    INTEGER NOT NULL,
  sp_def    INTEGER NOT NULL,
  speed     INTEGER NOT NULL,
  PRIMARY KEY (dex_no, form_code),
  FOREIGN KEY (dex_no) REFERENCES pokemon(id) ON UPDATE CASCADE ON DELETE CASCADE
) 

FKs on form_stats_overrides:


In [43]:
# Recreate table with a proper FK to pokemon(id)
c.execute("PRAGMA foreign_keys = OFF")
c.execute("BEGIN")

# Keep a copy of any existing rows (if any)
c.execute("""
CREATE TABLE IF NOT EXISTS _form_stats_overrides_backup AS
SELECT * FROM form_stats_overrides
""")

# Drop and recreate with correct schema (PK on (dex_no, form_code) + FK to pokemon(id))
c.execute("DROP TABLE IF EXISTS form_stats_overrides")
c.execute("""
CREATE TABLE form_stats_overrides (
  dex_no    INTEGER NOT NULL,
  form_code TEXT    NOT NULL,
  hp        INTEGER NOT NULL,
  attack    INTEGER NOT NULL,
  defense   INTEGER NOT NULL,
  sp_atk    INTEGER NOT NULL,
  sp_def    INTEGER NOT NULL,
  speed     INTEGER NOT NULL,
  PRIMARY KEY (dex_no, form_code),
  FOREIGN KEY (dex_no) REFERENCES pokemon(id) ON UPDATE CASCADE ON DELETE CASCADE
)
""")

# (Optional) restore any rows that still satisfy the FK
c.execute("""
INSERT OR IGNORE INTO form_stats_overrides
(dex_no, form_code, hp, attack, defense, sp_atk, sp_def, speed)
SELECT dex_no, form_code, hp, attack, defense, sp_atk, sp_def, speed
FROM _form_stats_overrides_backup
""")

c.execute("DROP TABLE IF EXISTS _form_stats_overrides_backup")
c.execute("COMMIT")
c.execute("PRAGMA foreign_keys = ON")
conn.commit()


In [44]:
rows = []
for dex_id, (mega_name, stats) in megas_list:
    if dex_id is None or len(stats) != 6:
        continue
    form_code = "Mega X" if mega_name.endswith(" X") else "Mega Y" if mega_name.endswith(" Y") else "Mega"
    hp, atk, defe, spa, spd, spe = map(int, stats)
    rows.append((int(dex_id), form_code, hp, atk, defe, spa, spd, spe))

print("Prepared rows:", len(rows))

before = conn.total_changes
c.executemany("""
INSERT INTO form_stats_overrides
  (dex_no, form_code, hp, attack, defense, sp_atk, sp_def, speed)
VALUES (?,?,?,?,?,?,?,?)
ON CONFLICT(dex_no, form_code) DO UPDATE SET
  hp=excluded.hp,
  attack=excluded.attack,
  defense=excluded.defense,
  sp_atk=excluded.sp_atk,
  sp_def=excluded.sp_def,
  speed=excluded.speed
""", rows)
conn.commit()
print("Rows changed:", conn.total_changes - before)


Prepared rows: 74
Rows changed: 74
